In [28]:
import mysql.connector
import pandas as pd

In [69]:
# Establecer la conexión con la base de datos
conexion = mysql.connector.connect(
    host='localhost',
    user='root',
    password='1533542415',
    database='quantyle_analitics'
)

# Verificar si la conexión se realizó con éxito
if conexion.is_connected():
    print("Conexión exitosa")

# Realizar operaciones en la base de datos, por ejemplo, ejecutar una consulta SQL
cursor = conexion.cursor()

# Ejemplo de consulta SQL
consulta = "SELECT * FROM state"

# Ejecutar la consulta
cursor.execute(consulta)

# Obtener los resultados
resultados = cursor.fetchall()

# Mostrar los resultados
for fila in resultados:
    print(fila)

# Cerrar el cursor y la conexión
cursor.close()
conexion.close()


Conexión exitosa
(1, 'Florida')
(2, 'California')
(3, 'Illinois')
(4, 'New Jersey')


# 1) State

In [31]:
state = pd.read_parquet(r'..\..\..\datasets\processed\bd\states.parquet.gz')

# Establecer la conexión con la base de datos
conexion = mysql.connector.connect(
    host='localhost',
    user='root',
    password='1533542415',
    database='quantyle_analitics'
)

# Insertar filas en la tabla state
cursor = conexion.cursor()
for index, row in state.iterrows():
    cursor.execute("INSERT INTO state (state) VALUES (%s)", (row['state'],))  # Asegúrate de tener la coma al final

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


# 2) Categories

In [30]:
categories = pd.read_parquet(r'..\..\..\datasets\processed\bd\categories.parquet.gz')

# Establecer la conexión con la base de datos
conexion = mysql.connector.connect(
    host='localhost',
    user='root',
    password='1533542415',
    database='quantyle_analitics'
)

# Insertar filas en la tabla categories
cursor = conexion.cursor()
for index, row in categories.iterrows():
    cursor.execute("INSERT INTO categories (name) VALUES (%s)", (row['name'],))

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


# 3) User yelp

In [32]:
user_yelp = pd.read_parquet(r'..\..\..\datasets\processed\bd\user_yelp.parquet.gz')
user_yelp.drop(columns=['friends'],inplace=True)
user_yelp['creation'] = user_yelp['creation'].dt.strftime('%Y-%m-%d %H:%M:%S')


# Establecer la conexión con la base de datos
conexion = mysql.connector.connect(
    host='localhost',
    user='root',
    password='1533542415',
    database='quantyle_analitics'
)

# Insertar filas en la tabla bussiness_yelp
cursor = conexion.cursor()
for index, row in user_yelp.iterrows():
    cursor.execute("INSERT INTO user_yelp (user_id, name, creation, review_count, fans, stars) VALUES (%s, %s, %s, %s, %s, %s)",
                (row['user_id'], row['name'], row['creation'], row['review_count'], row['fans'], row['stars']))

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


# 4) User google

In [ ]:
user_google = pd.read_parquet(r'..\..\..\datasets\processed\bd\user_yelp.parquet.gz')
user_google.drop(columns=['friends'],inplace=True)
user_google['creation'] = user_google['creation'].dt.strftime('%Y-%m-%d %H:%M:%S')


# Establecer la conexión con la base de datos
conexion = mysql.connector.connect(
    host='localhost',
    user='root',
    password='1533542415',
    database='quantyle_analitics'
)

# Insertar filas en la tabla bussiness_yelp
cursor = conexion.cursor()
for index, row in user_yelp.iterrows():
    cursor.execute("INSERT INTO user_yelp (user_id, name, creation, review_count, fans, stars) VALUES (%s, %s, %s, %s, %s, %s)",
                (row['user_id'], row['name'], row['creation'], row['review_count'], row['fans'], row['stars']))

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


# Bussiness yelp

In [73]:
bussiness_google = pd.read_parquet(r'..\..\..\datasets\processed\bd\yelp.parquet.gz')
bussiness_google.rename(columns={'state': 'state_id'}, inplace=True)
bussiness_google["state_id"] = bussiness_google["state_id"].replace("Florida", 1)
bussiness_google["state_id"] = bussiness_google["state_id"].replace("California", 2)
bussiness_google["state_id"] = bussiness_google["state_id"].replace("Illinois", 3)
bussiness_google["state_id"] = bussiness_google["state_id"].replace("New Jersey", 4)
bussiness_google


,business_id,name,latitude,longitude,stars,state_id
0,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,27.955269,-82.456320,4.09,0
1,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,27.916116,-82.760461,4.45,0
2,uI9XODGY_2_ieTE6xJ0myw,Roman Forum,28.046203,-82.505053,4.00,0
3,JgpnXv_0XhV3SfbfB50nxw,Joe's Pizza,27.960514,-82.506127,4.17,0
4,pJfh3Ct8iL58NZa8ta-a5w,Top Shelf Sports Lounge,28.196252,-82.380615,4.51,0
...,...,...,...,...,...,...
12912,Df-DdXqssj7ZeD8ypAF7Og,Crux Cafe,39.894469,-75.071001,4.00,3
12913,qjtELTt9fdIwoi_xGNN21g,Chipotle Mexican Grill,39.943481,-74.964743,2.43,3
12914,BzPTw9daJW8ToqTc7QiHFw,Taco Bell,40.210771,-74.756588,2.67,3
12915,UBQAksw81m0sMrAd8g-ECg,El Zarape Restaurant,39.718717,-74.971143,4.00,3


In [75]:
bussiness_google_1 = bussiness_google.iloc[:20]
bussiness_google_1


,business_id,name,latitude,longitude,stars,state_id
0,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,27.955269,-82.456320,4.09,0
1,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,27.916116,-82.760461,4.45,0
2,uI9XODGY_2_ieTE6xJ0myw,Roman Forum,28.046203,-82.505053,4.00,0
3,JgpnXv_0XhV3SfbfB50nxw,Joe's Pizza,27.960514,-82.506127,4.17,0
4,pJfh3Ct8iL58NZa8ta-a5w,Top Shelf Sports Lounge,28.196252,-82.380615,4.51,0
5,RK6-cJ9hj53RzOlCBmpT-g,Impasto,27.890814,-82.502346,5.00,0
6,vje0KIiE7vtpx7JzmBx5LQ,The Pearl,27.769405,-82.767317,4.15,0
7,aNtKyc2rr-uK5cqzY9TVQQ,Chipotle Mexican Grill,27.894167,-82.779866,3.20,0
8,P10BsKUeoeCeVoK4BS2u0w,Trust Me BBQ,27.865257,-82.326090,3.94,0
9,QjV4v7q_pt7tt3K1US7IHg,PDQ Temple Terrace,28.054888,-82.399548,3.00,0


In [ ]:
# Establecer la conexión con la base de datos
conexion = mysql.connector.connect(
    host='localhost',
    user='root',
    password='1533542415',
    database='quantyle_analitics'
)

# Insertar filas en la tabla business_yelp por lotes
cursor = conexion.cursor()

# Convertir el DataFrame a una lista de tuplas para la inserción en lotes
data_to_insert = [tuple(row) for row in bussiness_google_1[['business_id', 'name', 'latitude', 'longitude', 'stars', 'state_id']].to_numpy()]

# Consulta de inserción
insert_query = "INSERT INTO business_yelp (business_id, name, latitude, longitude, stars, state_id) VALUES (%s, %s, %s, %s, %s, %s)"

# Insertar en lotes utilizando executemany()
cursor.executemany(insert_query, data_to_insert)

# Hacer commit de los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()


# Bussiness google